In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY310,
)

In [2]:
import os
import subprocess

import GooseSLURM as gs
import numpy as np

import lysis

In [3]:
experiment_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("file_code", np.str_, 40),
        ("forced_unbind", np.float_),
        ("average_bind_time", np.float_),
    ]
)
code_type = np.dtype([("file_code", np.str_, 40), ("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("experiment", np.str_, 40),
        ("code", np.str_, 40),
        ("seed", int),
        ("running_time", int),
    ]
)

In [4]:
experiments = np.array(
    [
        ("Physiological Kd", "", 8.52e-2, 27.8),
        ("10x smaller", "_Kd00020036", 0.5143, 277.8),
        ("10x bigger", "_Kd0236", 5.4e-3, 2.78),
    ],
    dtype=experiment_type,
)
programs = np.array(
    [
        ("_along", "Diffuse along clot", "macro_Q2_diffuse_along"),
        ("_always", "Always bind", "macro_Q2_always_rebind"),
        ("_into_and_along", "Diffuse into and along clot - BUGGED", "macro_Q2_diffuse_into_and_along"),
        (
            "_into_and_along_fixed",
            "Diffuse into and along clot - FIXED",
            "macro_Q2_diffuse_into_and_along_fixed",
        ),
        ("_into", "Diffuse into clot", "macro_Q2_diffuse_into"),
    ],
    dtype=code_type,
)
runs = np.empty(15, dtype=run_type)

In [5]:
in_file_code = "_PLG2_tPA01{data_code}_Q2.dat"
out_file_code = "_PLG2_tPA01{data_code}{program_code}_Q2.dat"

In [6]:
runs = np.array(
    [
        ("2023-01-30-2000", "Physiological Kd", "Diffuse along clot", 17109424, 1200),
        ("2023-01-30-2001", "Physiological Kd", "Always bind", 9965734, 1800),
        ("2023-01-30-2002", "Physiological Kd", "Diffuse into and along clot - BUGGED", -2137354075, 1200),
        ("2023-01-30-2003", "Physiological Kd", "Diffuse into and along clot - FIXED", -2137354075, 1800),
        ("2023-01-30-2004", "Physiological Kd", "Diffuse into clot", -2135977853, 1200),
        ("2023-01-30-2005", "10x smaller", "Diffuse along clot", -848304637, 1200),
        ("2023-01-30-2006", "10x smaller", "Always bind", 1299539472, 1800),
        ("2023-01-30-2007", "10x smaller", "Diffuse into and along clot - BUGGED", -854989241, 1200),
        ("2023-01-30-2008", "10x smaller", "Diffuse into and along clot - FIXED", -854989241, 1800),
        ("2023-01-30-2009", "10x smaller", "Diffuse into clot", -850336215, 1200),
        ("2023-01-30-2010", "10x bigger", "Diffuse along clot", -1216563743, 1200),
        ("2023-01-30-2011", "10x bigger", "Always bind", 669985532, 900),
        ("2023-01-30-2012", "10x bigger", "Diffuse into and along clot - BUGGED", -1212172957, 1200),
        ("2023-01-30-2013", "10x bigger", "Diffuse into and along clot - FIXED", -1212172957, 1800),
        ("2023-01-30-2014", "10x bigger", "Diffuse into clot", -1213352577, 1200),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    prog = programs[programs["descriptor"] == run["code"]]
    exper = experiments[experiments["descriptor"] == run["experiment"]]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    p = {
        "forced_unbind": float(exper["forced_unbind"]),
        "average_bind_time": float(exper["average_bind_time"]),
        "seed": int(run["seed"]),
        "total_time": int(run["running_time"]),
        "macro_version": "macro"
        + out_file_code[:-4].format(data_code=exper["file_code"][0], program_code=prog["file_code"][0]),
    }
    e.initialize_macro_param(p)
    # e.to_file()

In [36]:
for run in runs:
    prog = programs[programs["descriptor"] == run["code"]]
    exper = experiments[experiments["descriptor"] == run["experiment"]]
    e = lysis.util.Experiment(os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"), experiment_code=run["exp_code"])
    e.read_file()
    f = lysis.util.FortranMacro(
        exp=e,
        executable=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "bin", prog["executable"][0]),
        in_file_code=in_file_code.format(data_code=exper["file_code"][0]),
        out_file_code=out_file_code.format(data_code=exper["file_code"][0], program_code=prog["file_code"][0]),
    )
    comm = f.exec_command()
    comm += [">", os.path.join(e.os_path, "macro" + out_file_code[:-4].format(data_code=exper["file_code"][0], program_code=prog["file_code"][0]) + ".txt")]
    comm = " ".join(comm)
    # job-options
    sbatch = {
        "job-name": "lysis" + e.experiment_code,
        "out": os.path.join(e.os_path, "job.slurm-%j.out"),
        "nodes": 1,
        "ntasks": 1,
        "cpus-per-task": 1,
    }
    script = [
        "module purge",
        "module load oneapi/compiler",
        comm
    ]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-01-30-2000/job.slurm'], returncode=0, stdout=b'Submitted batch job 2214932\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-01-30-2001/job.slurm'], returncode=0, stdout=b'Submitted batch job 2214933\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-01-30-2002/job.slurm'], returncode=0, stdout=b'Submitted batch job 2214934\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-01-30-2003/job.slurm'], returncode=0, stdout=b'Submitted batch job 2214935\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-01-30-2004/job.slurm'], returncode=0, stdout=b'Submitted batch job 2214936\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2023-01-30-2005/job.slurm'], returncode=0, std

In [28]:
command = "/home/bpaynter/git/UCO-OpResearch/lysis/bin/macro_Q2_diffuse_along --expCode 2023-01-30-2000 --inFileCode _PLG2_tPA01_Q2.dat --outFileCode _PLG2_tPA01_along_Q2.dat --seed 17109424 > /home/bpaynter/git/UCO-OpResearch/lysis/data/2023-01-30-2000/macro_PLG2_tPA01_along_Q2.txt"
script = [
    "module purge",
    "module load oneapi/compiler",
    command
]
with open(os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "job.slurm"), "w") as file:
    file.write(gs.scripts.plain(script, **sbatch))

In [31]:
with open('slurm_test.out', 'w') as file:
    result = subprocess.run(
        ["sbatch", "job.slurm"],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        stdout=file,
#        stderr=STDOUT,
    )
print(result)

CompletedProcess(args=['sbatch', 'job.slurm'], returncode=0)
